<div class="alert alert-block alert-info" align="center">
    <h1>
        Imports
    </h1>
</div>

In [ ]:
import numpy as np
import pandas as pd
import mnist

from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical

<div class="alert alert-block alert-info" align="center">
    <h1>
        Déclarations
    </h1>
</div>

<div align="center"><h1> Fonctions utiles </h1></div>

In [ ]:
def plot_histories (eta, epochs, cost_history, accuracy_history):
    fig, ax = plt.subplots(figsize = (5, 5))
    ax.set_ylabel(r'$J(\theta)$')
    ax.set_xlabel('Epochs')
    ax.set_title(r"$\eta$ :{}".format(eta))
    line1, = ax.plot(range(epochs), cost_history, label = 'Cost')
    line2, = ax.plot(range(epochs), accuracy_history, label = 'Accuracy')
    plt.legend(handler_map = {line1: HandlerLine2D(numpoints = 4)})

def plot_decision_boundary(func, X, y):
    amin, bmin = X.min(axis = 0) - 0.1
    amax, bmax = X.max(axis = 0) + 0.1
    hticks = np.linspace(amin, amax, 101)
    vticks = np.linspace(bmin, bmax, 101)

    aa, bb = np.meshgrid(hticks, vticks)
    ab = np.c_[aa.ravel(), bb.ravel()]
    c = func(ab)
    cc = c.reshape(aa.shape)

    cm = plt.cm.RdBu
    cm_bright = ListedColormap(['#FF0000', '#0000FF'])

    fig, ax = plt.subplots()
    contour = plt.contourf(aa, bb, cc, cmap = cm, alpha = 0.8)

    ax_c = fig.colorbar(contour)
    ax_c.set_label("$P(y = 1)$")
    ax_c.set_ticks([0, 0.25, 0.5, 0.75, 1])

    plt.scatter(X[:, 0], X[:, 1], c = y, cmap = cm_bright)
    plt.xlim(amin, amax)
    plt.ylim(bmin, bmax)
    plt.title("Decision Boundary")

def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def sigmoid_prime(x):
    return sigmoid(x) * (1.0 - sigmoid(x))

def tanh(x):
    return np.tanh(x)

def tanh_prime(x):
    return 1 - x ** 2

def relu(x):
    return np.maximum(0, x)

def relu_prime(x):
    x[x <= 0] = 0
    x[x > 0] = 1
    return x

def leakyrelu(x):
    return np.maximum(0.01, x)

def leakyrelu_prime(x):
    x[x <= 0] = 0.01
    x[x > 0] = 1
    return x

def softmax(x):
    expx = np.exp(x - np.max(x))
    return expx / expx.sum(axis = 0, keepdims = True)

<div align="center"><h1> Classes </h1></div>

In [ ]:
class Conv3x3:
    # A Convolution layer using 3x3 filters.

    def __init__(self, num_filters):
        self.num_filters = num_filters
        
        print ("Creation d'un Conv3x3 avec ", num_filters)
        # filters is a 3d array with dimensions (num_filters, 3, 3)
        # We divide by 9 to reduce the variance of our initial values
        self.filters = np.random.randn(num_filters, 3, 3) / 9

    def iterate_regions(self, image):
        '''
        Generates all possible 3x3 image regions using valid padding.
        - image is a 2d numpy array
        '''
        h, w = image.shape

        for i in range(h - 2):
            for j in range(w - 2):
                im_region = image[i:(i + 3), j:(j + 3)]
                yield im_region, i, j

    def forward(self, input):
        '''
        Performs a forward pass of the conv layer using the given input.
        Returns a 3d numpy array with dimensions (h, w, num_filters).
        - input is a 2d numpy array
        '''
        #on doit faire ne sorte que l'entrée soit un tableau 3D non 2d
        print ("Je suis dans le forward de Conv3x3")
        self.last_input = input
        print(input.shape)
        #h, w = input.shape
        h=input.shape[0]
        w=input.shape[1]
        output = np.zeros((h - 2, w - 2, self.num_filters))

        for im_region, i, j in self.iterate_regions(input):
            output[i, j] = np.sum(im_region * self.filters, axis=(1, 2))

        return output
    
    def backprop(self, d_L_d_out, learn_rate):
        '''
        Performs a backward pass of the conv layer.
        - d_L_d_out is the loss gradient for this layer's outputs.
        - learn_rate is a float.
        '''
        d_L_d_filters = np.zeros(self.filters.shape)

        for im_region, i, j in self.iterate_regions(self.last_input):
          for f in range(self.num_filters):
            d_L_d_filters[f] += d_L_d_out[i, j, f] * im_region

        # Update filters
        self.filters -= learn_rate * d_L_d_filters

        # We aren't returning anything here since we use Conv3x3 as the first layer in our CNN.
        # Otherwise, we'd need to return the loss gradient for this layer's inputs, just like every
        # other layer in our CNN.
        return None
        
    
    

class MaxPool2:
    # A Max Pooling layer using a pool size of 2.

    def iterate_regions(self, image):
        '''
        Generates non-overlapping 2x2 image regions to pool over.
        - image is a 2d numpy array
        '''
        h, w, _ = image.shape
        new_h = h // 2
        new_w = w // 2

        for i in range(new_h):
            for j in range(new_w):
                im_region = image[(i * 2):(i * 2 + 2), (j * 2):(j * 2 + 2)]
                yield im_region, i, j

    def forward(self, input):
        '''
        Performs a forward pass of the maxpool layer using the given input.
        Returns a 3d numpy array with dimensions (h / 2, w / 2, num_filters).
        - input is a 3d numpy array with dimensions (h, w, num_filters)
        '''
        print ("je suis dans le forward de MaxPool2")
        self.last_input = input
        h, w, num_filters = input.shape
        output = np.zeros((h // 2, w // 2, num_filters))

        for im_region, i, j in self.iterate_regions(input):
            output[i, j] = np.amax(im_region, axis=(0, 1))

        return output
    
    def backprop(self, d_L_d_out):
        '''
        Performs a backward pass of the maxpool layer.
        Returns the loss gradient for this layer's inputs.
        - d_L_d_out is the loss gradient for this layer's outputs.
    '''
        d_L_d_input = np.zeros(self.last_input.shape)

        for im_region, i, j in self.iterate_regions(self.last_input):
          h, w, f = im_region.shape
          amax = np.amax(im_region, axis=(0, 1))

          for i2 in range(h):
            for j2 in range(w):
              for f2 in range(f):
                # If this pixel was the max value, copy the gradient to it.
                if im_region[i2, j2, f2] == amax[f2]:
                  d_L_d_input[i * 2 + i2, j * 2 + j2, f2] = d_L_d_out[i, j, f2]

        return d_L_d_input
    
class MyFlatten: # A Flattening layer
    def forward(self, input):
        print(f'input : {input.shape}')
        print(f'output : {input.flatten().shape}')
        self.last_input_shape = input.shape
        input = input.flatten()
        self.last_input = input
        return input
    def backprop(self, d_L_d_out):
        return d_L_d_out.reshape(self.last_input_shape)
    
class MyLayer:
    def __init__(self, *args, **kwargs):
        self.input = kwargs.get("input", None) # Number of neurons at layer i-1
        self.output = kwargs.get("output", None) # Number of neurons at  layer i (current layer) 
        self.activ_function_curr = kwargs.get("activation", None) # Activation function for the layer
        self.paramCouche = kwargs.get("paramCouche", None) # Param de la couche Conv3x3
        self.type = kwargs.get("type", None)
        self.couche = kwargs.get("couche", None)
        self.parameters = {}
        self.derivatives = {}
        self.activation_func = None
        #self.activationCNNFunc = None
        #self.outputCNN = None
        
        
        if self.activ_function_curr == "relu":
                self.activation_func = relu
                self.backward_activation_func = relu_prime
        elif self.activ_function_curr == "sigmoid":
                self.activation_func = sigmoid
                self.backward_activation_func = sigmoid_prime
        elif self.activ_function_curr == "tanh":
                self.activation_func = tanh
                self.backward_activation_func = tanh_prime
        elif self.activ_function_curr == "leakyrelu":
                self.activation_func = leakyrelu
                self.backward_activation_func = leakyrelu_prime
        elif self.activ_function_curr == "softmax":
                self.activation_func = softmax
                self.backward_activation_func = softmax
        
              
                
                
    def initParams(self):
        # Initialisation du dictionnaire de données parameters contenant W, A et Z pour un layer
        print ("Je suis dans init Params, là je mets des W et b avec des randoms")
        seed = 30
        np.random.seed(seed)
        self.parameters['W'] = np.random.randn(self.output, self.input) * np.sqrt(2 / self.input)
        self.parameters['b'] = np.random.randn(self.output, 1) * 0.1

    def setW(self, matW):
        self.parameters['W'] = np.copy(matW)
        
    def setA(self, matA):
        self.parameters['A'] = np.copy(matA) 
        
    def setZ(self, matZ):
        self.parameters['Z'] = np.copy(matZ)
    
    def setB(self, matB):
        self.parameters['b'] = np.copy(matB)
        
    def setdW(self, matdW):
        self.parameters['dW'] = np.copy(matdW)
        
    def setdA(self, matdA):
        self.parameters['dA'] = np.copy(matdA)
        
    def setdZ(self, matdZ):
        self.parameters['dZ'] = np.copy(matdZ)
    
    def setdB(self, matdB):
        self.parameters['db'] = np.copy(matdB)

class MyNeuralNetwork:
    def __init__(self):
        self.nbLayers = 0
        self.nbCNNlayers = 0
        self.layers = [] # NN layers
        #self.CNN = [] # CNN layers
        self.CNN_EXIST=False
        self.outFlat = 0

    def printLayers(self):
        for i in range(len(self.CNN)):
            print(self.CNN[i].activationCNNFunc)
        for i in range(len(self.layers)):
            print(self.layers[i].activation_func)
        
    def info(self):
        print(f'Content of the network :');
        j = 0;
        for i in range(len(self.CNN)):
            print(f'\n\tLayer n° {i} du CNN => ')
            print(f'\t\tInput : {self.CNN[i].input}\n\t\tOutput : {self.CNN[i].output}')
            if (i != 0):
                print(f'\t\tCouche : {self.CNN[i].activationCNNFunc}')
                print(f'\t\tW shape : {self.CNN[i].parameters["W"].shape}\n')
                #t\tW data :\n{self.CNN[i].parameters["W"]}')
                print(f'\t\tb shape : {self.CNN[i].parameters["b"].shape}\n')
                #\t\tb data :\n{self.CNN[i].parameters["b"]}')
                
        for i in range(len(self.layers)):
            print(f'\n\tLayer n° {i} du NN => ')
            print(f'\t\tInput : {self.layers[i].input}\n\t\tOutput : {self.layers[i].output}')
            if (i != 0):
                print(f'\t\tCouche : {self.layers[i].activation_func}')
                print(f'\t\tW shape : {self.layers[i].parameters["W"].shape}\n')
                #\t\tW data :\n{self.layers[i].parameters["W"]}')
                print(f'\t\tb shape : {self.layers[i].parameters["b"].shape}\n')
                #\t\tb data :\n{self.layers[i].parameters["b"]}')

    def addLayer(self, layer):
        self.nbLayers += 1 # Le layer 0 = input, si CNN = images
        print ("Layer",layer)
        if (type(layer) is Conv3x3):
            print ("c'est un 3x3")
            self.layers.append(layer)
            self.CNN_EXIST=True # il existe un CNN donc pour le RNN il faudra prendre la sortie du flatten
        if (type(layer) is MaxPool2):    
            print ("c'est un maxPool2")
            self.layers.append(layer)
        if (type(layer) is MyFlatten):    
            print ("c'est un Flatten")
            self.layers.append(layer)
        if (type(layer) is MyLayer):    
            print ("c'est un MyLayer") 
            self.layers.append(layer)           
            # attention s'il y a un CNN on n'a pas initialisé les valeurs
            # il faut le faire dans le predict avec un init_Params()
            if self.CNN_EXIST != True: # on est dans le cas normal où il n'y a pas de CNN
                if (self.nbLayers==1): 
                    # this is the first layer so adding a layer 0
                    layerZero=Layer(layer.input)
                    self.layers.append(layerZero)
            
                self.layers.append(layer) 
                self.layers[self.nbLayers].input=self.layers[self.nbLayers-1].output
                #self.layers[self.nbLayers].output=self.layers[self.nbLayers].output
                layer.initParams()
           #si on a un CNN, il faut récupérer la valeur du flatten pour le forward
                    
          

    def set_parametersW_b (self, numlayer, matX, matb):
        self.layers[numlayer].parameters['W'] = np.copy(matX)
        self.layers[numlayer].parameters['b'] = np.copy(matb)

    def forward_propagation(self, X):
        outConv=0
        outMaxPool=0
        outFlatten=0
        previousFlatten=False
        for num_layer in range (0, self.nbLayers):
            print ("\nPredict: lecture objet",self.layers[num_layer])
            if (type(self.layers[num_layer]) is Conv3x3):
                outConv=self.layers[num_layer].forward(X)
                print ("\tpasse dans Conv3x3")
            if (type(self.layers[num_layer]) is MaxPool2):
                outMaxPool=self.layers[num_layer].forward(outConv)
                print ("\tpasse dans MaxPool2")
            if (type(self.layers[num_layer]) is MyFlatten):
                outFlatten=self.layers[num_layer].forward(outMaxPool) 
                self.outFlat = outFlatten
                print ("\tpasse dans Flatten")
                previousFlatten=True
            if (type(self.layers[num_layer]) is MyLayer):
                # je viens de passer dans le flatten,récupération de l'input
                if (previousFlatten): # le précédent est flatten
                    print ("len Flatten", len(outFlatten))
                    #self.layers[num_layer].input = self.layers[self.nbLayers - 1].output
                    #layer.initParams()
                    #il faut initialiser les valeurs des paramètres
                    print ("Init des paramètres. Output = ",self.layers[num_layer].output)
                    print ("Le input doit prendre le flatten")
                    self.layers[num_layer].input=len(outFlatten)
                    print ("On relance init Param pour créer les W et b")
                    self.layers[num_layer].initParams()
                    self.layers[num_layer].setZ(np.dot(self.layers[num_layer].parameters['W'], 
                                       outFlatten) + self.layers[num_layer].parameters['b'])
                    self.layers[num_layer].setA(self.layers[num_layer].activation_func(self.layers[num_layer].parameters['Z']))
                    print("self.nbLayers - activation",self.layers[num_layer].activation_func)
                    previousFlatten=False
                else: # là le précédent n'est plus flatten je suis dans le RNN
                    print ("ICI je suis dans un RNN qui n'est pas après un flatten")
                    # il faut initialiser les valeurs de W et b
                    print ("Init des paramètres. Output = ",self.layers[num_layer].output)
                    print ("Le input doit prendre le layer precedent",self.layers[num_layer-1].output)
                    self.layers[num_layer].input=self.layers[num_layer-1].output
                    self.layers[num_layer].initParams()
                    self.layers[num_layer].setZ(np.dot(self.layers[num_layer].parameters['W'], 
                                       self.layers[num_layer - 1].parameters['A']) + self.layers[num_layer].parameters['b'])
                    # Applying the activation function of the layer to Z
                    self.layers[num_layer].setA(self.layers[num_layer].activation_func(self.layers[num_layer].parameters['Z']))
                    #self.activation_func = softmax
                    print("self.nbLayers - activation",self.layers[num_layer].activation_func)
                    print ("Taille A ",self.layers[num_layer].parameters['A'].shape)
        
        
    def cost_function(self, y):
        print("cost_function")
        return (-(y * np.log(self.layers[self.nbLayers-1].parameters['A'] + 1e-8) + (1 - y) * np.log(1 - self.layers[self.nbLayers-1].parameters['A'] + 1e-8))).mean()

    def backward_propagation(self, y):
        outConv=0
        outMaxPool=0
        outFlatten=0
        suivantFlatten=False
        if(type(self.layers[self.nbLayers-1]) is MyLayer):
            #calcul de dZ dW et db pour le dernier layer
            print("self.layers[self.nbLayers].parameters['A']",self.layers[self.nbLayers-1].parameters['A'])
            print("y",y)
            self.layers[self.nbLayers-1].derivatives['dZ']=self.layers[self.nbLayers-1].parameters['A']-y
            self.layers[self.nbLayers-1].derivatives['dW']=np.dot(self.layers[self.nbLayers-1].derivatives['dZ'],
                                                                 (self.layers[self.nbLayers-1].parameters['A']))
            m=self.layers[self.nbLayers-1].parameters['A'].shape[1]#égal au nombre de colonnes de A 
            self.layers[self.nbLayers-1].derivatives['db']=np.sum(self.layers[self.nbLayers-1].derivatives['dZ'], 
                                                           axis=1, keepdims=True) / m

        #calcul de dZ dW db pour les autres layers
        for l in range(self.nbLayers-2,0,-1) :
            if(type(self.layers[l]) is MyLayer):
                if(type(self.layers[l-1]) is MyFlatten):
                    self.layers[l].derivatives['dZ']=np.dot(np.transpose(self.layers[l+1].parameters['W']),
                                                    self.layers[l+1].derivatives['dZ'])*self.layers[l].backward_activation_func(self.layers[l].parameters["Z"])

                    self.layers[l].derivatives["dW"]=np.dot(self.layers[l].derivatives['dZ'],self.outFlat)

                    m=self.layers[l-1].parameters['A'].shape[1]#égal au nombre de colonnes de A 
                    self.layers[l].derivatives['db']=np.sum(self.layers[l].derivatives['dZ'], 
                                                               axis=1, keepdims=True) / m  


                    outputRNN=self.layers[l].parameters['A']
                    print("self.layers[l].parameters['A']",self.layers[l].parameters['A'])
                    suivantFlatten = True
                else :
                    elf.layers[l].derivatives['dZ']=np.dot(np.transpose(self.layers[l+1].parameters['W']),
                                                    self.layers[l+1].derivatives['dZ'])*self.layers[l].backward_activation_func(self.layers[l].parameters["Z"])

                    self.layers[l].derivatives["dW"]=np.dot(self.layers[l].derivatives['dZ'],
                                                    (self.layers[l-1].parameters['A']))

                    m=self.layers[l-1].parameters['A'].shape[1]#égal au nombre de colonnes de A 
                    self.layers[l].derivatives['db']=np.sum(self.layers[l].derivatives['dZ'], 
                                                               axis=1, keepdims=True) / m
            if(type(self.layers[l-1]) is MyFlatten):
                outFlatten=self.layers[l].backprop(outputRNN)
                suivantFlatten = False
            if(type(self.layers[l-1]) is MaxPool2):
                outMaxPool=self.layers[l].backprop(outFlatten)
            if(type(self.layers[l-1]) is Conv3x3):
                outConv=self.layers[l].backprop(outMaxPool)
        
                      
    def update_parameters(self, eta) :
        for l in range(1,self.nbLayers+1) :
            self.layers[l].parameters['W']-=eta*self.layers[l].derivatives['dW']
            self.layers[l].parameters["b"]-=eta*self.layers[l].derivatives["db"]

    def convert_prob_into_class(self,probs):
        probs = np.copy(probs)#pour ne pas perdre probs, i.e. y_hat
        probs[probs > 0.5] = 1
        probs[probs <= 0.5] = 0
        return probs

    def plot_W_b_epoch (self,epoch,parameter_history):
        mat=[]
        max_size_layer=0
        for l in range(1, self.nbLayers+1):    
            value=parameter_history[epoch]['W'+str(l)]
            if (parameter_history[epoch]['W'+str(l)].shape[1]>max_size_layer):
                max_size_layer=parameter_history[epoch]['W'+str(l)].shape[1]
            mat.append(value)
        figure=plt.figure(figsize=((self.nbLayers+1)*3,int (max_size_layer/2)))    
        for nb_w in range (len(mat)):    
                plt.subplot(1, len(mat), nb_w+1)
                plt.matshow(mat[nb_w],cmap = plt.cm.gist_rainbow,fignum=False, aspect='auto')
                plt.colorbar()    
        thelegend="Epoch "+str(epoch)
        plt.title (thelegend)    

    def accuracy(self,y_hat, y):
        if self.layers[self.nbLayers-1].activation_func==softmax:
            # si la fonction est softmax, les valeurs sont sur différentes dimensions
            # il faut utiliser argmax avec axis=0 pour avoir un vecteur qui indique
            # où est la valeur maximale à la fois pour y_hat et pour y
            # comme cela il suffit de comparer les deux vecteurs qui indiquent 
            # dans quelle ligne se trouve le max
            y_hat_encoded=np.copy(y_hat)
            y_hat_encoded = np.argmax(y_hat_encoded, axis=0)
            y_encoded=np.copy(y)
            y_encoded=np.argmax(y_encoded, axis=0)
            return (y_hat_encoded == y_encoded).mean()
        # la dernière fonction d'activation n'est pas softmax.
        # par exemple sigmoid pour une classification binaire
        # il suffit de convertir la probabilité du résultat en classe
        y_hat_ = self.convert_prob_into_class(y_hat)
        return (y_hat_ == y).all(axis=0).mean()       

    def predict(self, x):
        self.forward_propagation(x)
        return self.layers[self.nbLayers-1].parameters['A']

    def next_batch(self,X, y, batchsize):
        # pour avoir X de la forme : 2 colonnes, m lignes (examples) et également y
        # cela permet de trier les 2 tableaux avec un indices de permutation       
        X=np.transpose(X)
        y=np.transpose(y)
        
        m=len(y)
        print ("m ds next_batch",m)
        # permutation aléatoire de X et y pour faire des batchs avec des valeurs au hasard
        indices = np.random.permutation(m)
        X = X[indices]
        y = y[indices]
        for i in np.arange(0, X.shape[0], batchsize):
            # creation des batchs de taille batchsize
            yield (X[i:i + batchsize], y[i:i + batchsize])

    def fit(self, X, y, *args,**kwargs):    
        epochs=kwargs.get("epochs",20)
        verbose=kwargs.get("verbose",False)
        eta =kwargs.get("eta",0.01)
        batchsize=kwargs.get("batchsize",32)
        print ("Dans fit, X.shape, y.shape",X.shape,len(X),y.shape,y[0:2])
        #def fit(self, X, y, epochs, eta = 0.01,batchsize=64) :
        # sauvegarde historique coût et accuracy pour affichage
        cost_history = []
        accuracy_history = []
        parameter_history = []
        for i in range(epochs):
            i+=1
            # sauvegarde des coûts et accuracy par mini-batch
            cost = []
            accuracy = []
            ##########
            # on ne va passer qu'une image à la fois
            for nb_x in range(y.shape[0]):
                self.forward_propagation(X[nb_x])
                self.backward_propagation(y[nb_x])
                #self.update_parameters(eta)
                #out=self.layers[self.nbLayers-1].parameters['A']
                #loss = -np.log(out[y[nb_x]])
                #acc = 1 if np.argmax(out) == y[nb_x] else 0
                print (nb_x," calcul du cout pour ")
                print (y[nb_x])
                current_cost=self.cost_function(y[nb_x])
                print ("\t\tcurrent_cost",current_cost)
                
                cost.append(current_cost)
                y_hat = self.predict(X[nb_x])
                current_accuracy = self.accuracy(y_hat,y[nb_x])
                print ("\t\tPour y[",nb_x,"]",y[nb_x]," valeur prédite",y_hat)
                
                ############## TODO ###########
                # Revoir si on récupère bien l'accuracy 
                # Faire la backpropagation + update parameters
                    # Extraction et traitement d'un batch à la fois
                '''
                # mise en place des données au bon format
                batchX=np.transpose(batchX)
                if self.layers[self.nbLayers-1].activation_func==softmax:
                    # la classification n'est pas binaire, y a utilisé one-hot-encoder
                    # le batchy doit donc être transposé et le résultat doit
                    # être sous la forme d'une matrice de taille batchy.shape[1]
                    batchy=np.transpose(batchy.reshape((batchy.shape[0], batchy.shape[1])))
                    print("self.layers[self.nbLayers].activation_func==softmax")
                    print("batchy.shape[0]",batchy.shape[0],batchy.shape[1])
                else:
                    # il s'agit d'une classification binaire donc shape[1] n'existe pas
                    batchy=np.transpose(batchy.reshape((batchy.shape[0], 1)))
                    print("else self.layers[self.nbLayers].activation_func==softmax")
                #batchy=np.transpose(batchy.reshape((batchy.shape[0], 1)))
                print ("batchy",batchy)
                self.forward_propagation(batchX)
                #self.backward_propagation(batchy)
                #self.update_parameters(eta)
                '''
                      
                # sauvegarde pour affichage

                
                accuracy.append(current_accuracy)
                
            
            
            
            ##############
            
                
            #parameter_history.append(save_values)        
            #sauvegarde de la valeur moyenne des coûts et de l'accuracy du batch pour affichage
            current_cost=np.average(cost)
            cost_history.append(current_cost)
            current_accuracy=np.average(accuracy)
            accuracy_history.append(current_accuracy)
        
            if(verbose == True):
                print(f'Epoch : {i}/{epochs} | cost : {float(current_cost)}| accuracy : {current_accuracy}')
              
        return self.layers, cost_history, accuracy_history, parameter_history

<div class="alert alert-block alert-info" align="center">
    <h1>
        Applications
    </h1>
</div>

<div align="center"><h1> Classification des données de mnist </h1></div>

## Atemp 1

In [ ]:
test_images = mnist.test_images()[:10]
test_labels = mnist.test_labels()[:10]


# mis au bon format des images
newTestImages = []
for image in test_images:
    newTestImages.append(((image / 255) - 0.5))

X = np.float64(newTestImages)

#transformation des y en tableau binaire
y = to_categorical(test_labels, num_classes=10)   
print ("transformation des y pour etre au bon format", y.shape)
print ("exemple des donnees de y",y[0:3])

# Jeu d'apprentissage 60%
validation_size = 0.5

# 40% du jeu de données pour le test
testsize = 1 - validation_size

seed = 30

# séparation jeu d'apprentissage et jeu de test
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size = validation_size, random_state = seed, test_size = testsize)

# La transposée de X_train est de la forme : m colonnes (exemples), n lignes (nombre de variables prédictives)
#X_train=np.transpose(X_train)
print ("X_train.shape, y_train.shape", X_train.shape,y_train.shape)
# y_train est forcé pour être un tableau à 1 ligne contenant m colonnes
#y_train=np.transpose(y_train.reshape((y_train.shape[0], y_train.shape[1])))

# mêmes traitements pour le jeu de test
#X_test=np.transpose(X_test)
#y_test=np.transpose(y_test.reshape((y_test.shape[0], y_test.shape[1])))

# ATTENTION DANS LE MODELE COMME ON PASSE UNE IMAGE A LA FOIS IL FAUT QUE LE OUTPUT soit à 1
# cf ci-dessous
network = MyNeuralNetwork()
network.addLayer(Conv3x3(8))
network.addLayer(MaxPool2())
network.addLayer(MyFlatten())
network.addLayer(MyLayer(output = 1, activation = "relu"))
network.addLayer(MyLayer(output = 10, activation = "softmax"))

# le network info ne fonctionne pas je ne l'ai pas mis à jour
#network.info()

epochs = 5
eta = 0.01
batchsize=20

print(f'-------------------------------------------------------------------------')
print(f'shape de X_train')
print(X_train.shape)
print(f'shape de X_test')
print(X_test.shape)
print(f'shape de y_train')
print(y_train.shape)
print (y_train)
print(f'shape de y_test')
print(y_test.shape)
print(f'-------------------------------------------------------------------------')

#Entraînement du classifieur
layers,cost_history,accuracy_history,parameter_history=network.fit(X_train, y_train, verbose=True, epochs=epochs)

##### Attention les fonctions en dessous ne marcheront pas car je n'ai pas sauvegardé l'historique
# pour la prédiction il faut s'inspirer de y_hat = self.predict(X[nb_x])
# pour l'instant predict attend un tableau, là il faut passer valeur par valeur
#Prédiction
for i in range (y_test.shape[0]):
    y_pred=network.predict(X_test[i])
    accuracy_test = network.accuracy(y_pred, y_test[i])
    print("Accuracy test: %.3f"%accuracy_test)
#y_pred=network.predict(X_test)
#accuracy_test = network.accuracy(y_pred, y_test)
#print("Accuracy test: %.3f"%accuracy_test)

# Affichage des historiques
#plot_histories (eta,epochs,cost_history,accuracy_history)